In [5]:
import tensorflow as tf
import numpy as np
import os

from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
import matplotlib.pyplot as plt
# EarlyStopping : 
# Model Chechpoint : 

from preprocess import * 

In [6]:
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_' + string])
    plt.legend([string], 'val_' + string)

In [11]:
# 데이터 저장

DATA_IN_PATH = 'data_in/'
DATA_OUT_PATH = 'data_out/'

TRAIN_INPUTS = 'train_inputs.npy'
TRAIN_OUTPUTS = 'train_outputs.npy'
TRAIN_TARGETS = 'train_targets.npy'

DATA_CONFIGS = 'data_config.json'

In [12]:
tf.random.set_seed(1234)

In [13]:
index_inputs = np.load(open(DATA_IN_PATH + TRAIN_INPUTS, 'rb'))
index_outputs = np.load(open(DATA_IN_PATH + TRAIN_OUTPUTS, 'rb'))
index_targets = np.load(open(DATA_IN_PATH + TRAIN_TARGETS, 'rb'))

prepro_configs = json.load(open(DATA_IN_PATH + DATA_CONFIGS, 'r'))

In [16]:
print(len(index_inputs), len(index_outputs), len(index_targets))

11823 11823 11823


In [17]:
MODEL_NAME = 'seq2seq_kor'
BATCH_SIZE = 2
MAX_SEQUENCE = 25
EPOCH = 50
UNITS = 1024
EMBEDDING_DIM = 256
VALIDATION_SPLIT = 0.1

In [18]:
prepro_configs.keys()

dict_keys(['char2idx', 'idx2char', 'vocab_size', 'pad_symbol', 'std_symbol', 'end_symbol', 'unk_symbol'])

In [19]:
char2idx = prepro_configs['char2idx']
idx2char = prepro_configs['idx2char']
std_symbol = prepro_configs['std_symbol']
end_symbol = prepro_configs['end_symbol']
unk_symbol = prepro_configs['unk_symbol']

In [20]:
class Encoder(tf.keras.layers.Layer):
    def __init__(self, batch_sz):
        super(Encoder, self).__init__()        # 상속 와 이렇게 쓰는거구나...
        self.batch_sz = batch_sz
        self.enc_units = enc_units
        self.embedding_dim = embedding_dim
        self.vocab_size = vocab_size
        
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = tf.keras.layers.GRU(self.enc_units, return_sequences=True, return_state=True,
                                      recurrent_initializer = 'glorot_uniform')
    
    
    def call(self):
        x = self.embedding(x)
        self.gru(x, initial_state = hidden)
        
        return
    
    def initialize_hidden_state(self, ):
        return tf.zeros(tf.shape(inp)[0], self.enc_units)
        

In [21]:
class BahdanauAttention(tf.keras.layers.Layer):
    def __init__(self, ):
        super(BahdanauAttention, self).__init__()
        self.W1 = tf.keras.layers.Dense(units)
        self.W2 = tf.keras.layers.Dense(units)
        self.V = tf.keras.layers.Dense(1)
        
    def call(self, query, value):
        hidden_with_time_axis = tf.expand_dims(query, 1)
        score = self.W1(values) + self.W2(hidden_with_time_axis) # W1 : value에 관련된.. W2 : 쿼리에 관련된 것
        
        attention_weights = tf.nn.softmax(score, axis=1)
        
        context_vector = attention_weights * values
        context_vector = tf.reduce_sum(context_vector, axis = 1)
        
        return context_vector, attention_weights
        

In [23]:
# 오... 층을 아래에서부터 쌓는 것도 좋네
# 그러면 원하는 최종 결과물이 무엇인지를 보고, 그 결과가 나올 수 있게끔 필요한 것들과 양을 더 쉽게 정할 수 있을 듯

class Decoder(tf.keras.layers.Layer):
    def __init__(self, ):
        super(Decoder, self).__init__()
        
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim
        self.dec_units = dec_units
        self.batch_sz = batch_sz
        
        self.embedding = tf.keras.layers.Embedding(self.vocab_size, self.embedding_dim)
        self.gru = tf.keras.layers.GRU(self.dec_units, return_sequences = True, return_state = True,
                                      recurrent_initializer = 'glorot_uniform')
        self.fc = tf.keras.layers.Dense(self.vocab_size)
        self.attention = BahdanauAttention(self.dec_units)
        
    def call(self, x, enc_output):
        context_vector, attention_weights = self.attention(hidden, enc_output)
        x = self.embedding(x)
        tf.concat([tf.expand_dims(context_vector, 1), x], axis = -1)
        output, state = self.gru(x)
        
        output = tf.reshape(output, (-1, output.shape[2]))
        
        x = self.fc(output)
        
        return x, state, attention_weights